# Odometer Consistency Test Notebook
Tests Instagram scraping with odometer monitoring and intelligent fallbacks

In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

print('📦 Libraries loaded')

In [ ]:
# Initialize Selenium
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Keep visible for debugging
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(options=chrome_options)
print('🌐 Browser initialized')

In [ ]:
def convert_string_to_number(s):
    s = str(s).lower().strip()
    if ',' in s:
        return int(s.replace(',', ''))
    elif 'k' in s:
        return int(float(s.replace('k', '')) * 1000)
    elif 'm' in s:
        return int(float(s.replace('m', '')) * 1000000)
    elif 'b' in s:
        return int(float(s.replace('b', '')) * 1000000000)
    else:
        try:
            return int(float(s))
        except:
            return 0

In [ ]:
class InstagramOdometerTester:
    def __init__(self, username):
        self.username = username
        self.follower_count = 0
        self.readings = []
        
    def test_livecounts_odometer(self, max_attempts=5, wait_between=3):
        """Monitor LiveCounts odometer until stable"""
        print(f"\n🔍 Testing LiveCounts.nl Odometer (Max {max_attempts} attempts)")
        url = f'https://livecounts.nl/instagram-realtime/?u={self.username}'
        
        try:
            driver.get(url)
            time.sleep(3)  # Initial load
            
            readings = []
            for attempt in range(max_attempts):
                try:
                    el = driver.find_element(By.CSS_SELECTOR, '.odometer-inside')
                    txt = el.text
                    val = convert_string_to_number(re.sub(r'[^0-9]', '', txt))
                    
                    # Sanity check
                    if 0 < val < 700000000:
                        readings.append(val)
                        print(f"  Attempt {attempt+1}: {val:,} ✅")
                        
                        # Check for stability (last 2 readings within 0.1%)
                        if len(readings) >= 2:
                            diff = abs(readings[-1] - readings[-2])
                            pct = (diff / readings[-1]) * 100
                            if pct < 0.1:  # Stable within 0.1%
                                print(f"  ✅ STABLE! Variance: {pct:.3f}%")
                                self.follower_count = readings[-1]
                                return True
                    else:
                        print(f"  Attempt {attempt+1}: {val:,} ❌ (Out of range)")
                except Exception as e:
                    print(f"  Attempt {attempt+1}: Error - {e}")
                
                if attempt < max_attempts - 1:
                    time.sleep(wait_between)
            
            # If we got here, pick the most common value
            if readings:
                from collections import Counter
                most_common = Counter(readings).most_common(1)[0][0]
                self.follower_count = most_common
                print(f"  ⚠️ Used most common value: {most_common:,}")
                return True
                
        except Exception as e:
            print(f"  ❌ LiveCounts failed: {e}")
        return False
    
    def test_instastatistics_odometer(self, max_attempts=5, wait_between=3):
        """Monitor InstaStatistics odometer until stable"""
        print(f"\n🔍 Testing InstaStatistics Odometer (Max {max_attempts} attempts)")
        url = f'https://instastatistics.com/{self.username}'
        
        try:
            driver.get(url)
            time.sleep(3)
            
            readings = []
            for attempt in range(max_attempts):
                try:
                    el = driver.find_element(By.CSS_SELECTOR, '.odometer-inside')
                    txt = el.text
                    val = convert_string_to_number(re.sub(r'[^0-9]', '', txt))
                    
                    if 0 < val < 700000000:
                        readings.append(val)
                        print(f"  Attempt {attempt+1}: {val:,} ✅")
                        
                        if len(readings) >= 2:
                            diff = abs(readings[-1] - readings[-2])
                            pct = (diff / readings[-1]) * 100
                            if pct < 0.1:
                                print(f"  ✅ STABLE! Variance: {pct:.3f}%")
                                self.follower_count = readings[-1]
                                return True
                    else:
                        print(f"  Attempt {attempt+1}: {val:,} ❌ (Out of range)")
                except Exception as e:
                    print(f"  Attempt {attempt+1}: Error - {e}")
                
                if attempt < max_attempts - 1:
                    time.sleep(wait_between)
            
            if readings:
                from collections import Counter
                most_common = Counter(readings).most_common(1)[0][0]
                self.follower_count = most_common
                print(f"  ⚠️ Used most common value: {most_common:,}")
                return True
                
        except Exception as e:
            print(f"  ❌ InstaStatistics failed: {e}")
        return False
    
    def test_instagram_api(self):
        """Try Instagram API"""
        print(f"\n🔍 Testing Instagram API")
        try:
            url = f'https://i.instagram.com/api/v1/users/web_profile_info/?username={self.username}'
            h = {'User-Agent': 'Mozilla/5.0', 'x-ig-app-id': '936619743392459'}
            r = requests.get(url, headers=h, timeout=10)
            if r.status_code == 200:
                count = r.json()['data']['user']['edge_followed_by']['count']
                print(f"  Result: {count:,} ✅")
                self.follower_count = count
                return True
            else:
                print(f"  Status: {r.status_code} ❌")
        except Exception as e:
            print(f"  ❌ API failed: {e}")
        return False
    
    def test_instagram_meta(self):
        """Scrape Instagram meta tag"""
        print(f"\n🔍 Testing Instagram Meta Tag")
        try:
            driver.get(f'https://www.instagram.com/{self.username}/')
            time.sleep(5)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            meta = soup.find('meta', attrs={'property': 'og:description'})
            if meta:
                content = meta.get('content', '')
                print(f"  Meta content: {content[:100]}...")
                match = re.search(r'([\d,.]+[KMB]?)\s*Followers', content, re.I)
                if match:
                    val = convert_string_to_number(match.group(1))
                    if 0 < val < 2000000000:
                        print(f"  Result: {val:,} ✅")
                        self.follower_count = val
                        return True
        except Exception as e:
            print(f"  ❌ Meta tag failed: {e}")
        return False
    
    def run_all_tests(self):
        """Run all tests in order until one succeeds"""
        print(f"\n{'='*60}")
        print(f"🧪 TESTING INSTAGRAM SCRAPING FOR @{self.username}")
        print(f"{'='*60}")
        
        methods = [
            ("API", self.test_instagram_api),
            ("LiveCounts Odometer", self.test_livecounts_odometer),
            ("InstaStatistics Odometer", self.test_instastatistics_odometer),
            ("Instagram Meta Tag", self.test_instagram_meta)
        ]
        
        for name, method in methods:
            if method():
                print(f"\n✅ SUCCESS with {name}: {self.follower_count:,} followers")
                return self.follower_count
        
        print("\n❌ All methods failed")
        return 0

In [ ]:
# Test with Taylor Swift
tester = InstagramOdometerTester("taylorswift")
result = tester.run_all_tests()

print(f"\n{'='*60}")
print(f"📊 FINAL RESULT")
print(f"{'='*60}")
print(f"Username: @{tester.username}")
print(f"Followers: {result:,}")
print(f"Expected: ~281,000,000")
if result > 0:
    diff_pct = abs(result - 281000000) / 281000000 * 100
    print(f"Difference: {diff_pct:.2f}%")
    if diff_pct < 5:
        print("✅ Result is accurate!")
    else:
        print("⚠️ Result may be inaccurate")

In [ ]:
# Test with another artist
tester2 = InstagramOdometerTester("selenagomez")
result2 = tester2.run_all_tests()

print(f"\n{'='*60}")
print(f"📊 FINAL RESULT")
print(f"{'='*60}")
print(f"Username: @{tester2.username}")
print(f"Followers: {result2:,}")

In [ ]:
# Cleanup
driver.quit()
print('🛑 Browser closed')